# FAISS 관리

## FAISS 저장시 생기는 파일
### 1. **.faiss 파일**
- **내용**: `.faiss` 파일은 FAISS 인덱스를 저장. 벡터 데이터에 대한 인덱스 구조와 해당 데이터를 검색하기 위한 최적화된 인덱스를 포함.
- **용도**: 인덱스를 다시 로드하여 검색 작업을 수행할 수 있다. 이 파일만 있으면 벡터 데이터에 대한 검색 작업을 다시 수행할 수 있다.

### 2. **.pkl 파일**
- **내용**: `.pkl` 파일은 일반적으로 Python에서 사용하는 피클(pickle) 파일 형식으로 주로 벡터 데이터의 메타정보 또는 인덱스와 관련된 추가 데이터가 저장. 예를 들어, 벡터에 대한 ID, 메타데이터, 인덱스 구성 설정 등이 포함된다.
- **용도**: `.pkl` 파일은 Python 객체를 직렬화하여 저장한 것으로, 인덱스를 다시 로드할 때 사용된다. 이를 통해 검색 과정에서 필요한 부가적인 정보나 설정을 복원할 수 있다.

### 요약
- **.faiss 파일**: 벡터 인덱스 데이터를 저장하는 파일로, 벡터 검색 작업에 직접 사용.
- **.pkl 파일**: Python 객체의 직렬화된 데이터를 저장하는 파일로, 인덱스와 관련된 메타정보나 설정을 포함.

이 파일들은 함께 사용되어 벡터 검색 작업을 빠르게 수행할 수 있도록 한다. 인덱스를 로드할 때 두 파일을 모두 사용하여 완전한 검색 환경을 복원할 수 있다.

In [2]:
# 한번만 수행하면 됨
# 필요한 라이브러리 설치
# !pip install chardet

# 변수 선언

In [20]:
# 경로 추가
import sys
import os
from langchain.embeddings import OllamaEmbeddings
sys.path.append(os.path.dirname(os.getcwd()))

# 변수 선언
ROOT_PATH = '/Users/passion1014/project/langchain/rag_server'
INDEX_ID = "news01" # VectorDB INDEX 명
FILE_NAME = f'{INDEX_ID}.txt' # 파일이름

# 임베딩 선언
embeddings = OllamaEmbeddings( # embeddings = OpenAIEmbeddings()
    model="nomic-embed-text",
    base_url="http://localhost:11434"  # Ollama의 기본 URL입니다. 필요에 따라 변경하세요.
)


# 데이터 저장

In [51]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import chardet
import argparse
from tools import upload_vectordb

# upload_vectordb.main(f'{ROOT_PATH}/data/{INDEX_ID}.txt', INDEX_ID)

# 파일 내용 로드 및 처리
text = upload_vectordb.load_and_detect_encoding(f'{ROOT_PATH}/data/{INDEX_ID}.txt')


# 2. 의미별로 chunk로 나누기
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10)
chunks = text_splitter.split_text(text)

# 각 chunk를 문서 객체로 변환
docs = text_splitter.create_documents(chunks)

# FAISS 인덱스 생성 및 문서 추가
db = FAISS.from_documents(docs, embeddings)

# FAISS 인덱스 저장
        
db.save_local(f'{ROOT_PATH}/vectordb/{INDEX_ID}')
print(f"FAISS 인덱스를 '{vectordb_path}/{INDEX_ID}'에 저장했습니다.")



Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 184, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 138, which is longer than the specified 100
Created a chunk of size 202, which is longer than the specified 100
Created a chunk of size 145, which is longer than the specified 100
Created a chunk of size 157, which is longer than the specified 100
Created a chunk of size 147, which is longer than the specified 100
Created a chunk of size 130, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 128, which is longer than the specified 100
Created a chunk of size 220, which is longer than the specified 100
Created a chunk of size 127, which is longer than the specified 100
Created a chunk of size 137, which is longer tha

FAISS 인덱스를 '/Users/passion1014/project/langchain/rag_server/vectordb/news01'에 저장했습니다.


# 저장된 파일 확인

In [56]:
import pickle
import faiss

# 파일 경로
pkl_file_path = f'{ROOT_PATH}/vectordb/{INDEX_ID}/index'


# .pkl 파일 읽기
with open(f'{pkl_file_path}.pkl', 'rb') as f:
    docstore, uuid_mapping = pickle.load(f)

# 데이터를 출력하여 확인
print(f'# pkl [docstore] = {docstore}')
print(f'# pkl [uuid_mapping][{len(uuid_mapping)}] = {uuid_mapping}')


# .faiss 파일 읽기
index = faiss.read_index(f'{pkl_file_path}.faiss')

# 인덱스 정보 출력
print(f'# 인덱스 정보 출력 = {index}')

# docstore 객체의 메서드와 속성 확인
# print(dir(docstore))


# UUID로 데이터를 조회
for i in uuid_mapping:
    document = docstore.search(uuid_mapping[i])
    print(f'# {uuid_mapping[i]} = {document}')


# pkl [docstore] = <langchain_community.docstore.in_memory.InMemoryDocstore object at 0x129e02e40>
# pkl [uuid_mapping][22] = {0: 'ce6e5b12-82ab-4017-9dc9-10d3358eb85c', 1: '8531c56d-c487-421e-ba9f-54415a8560c1', 2: 'e618ea28-a04c-49e0-b6fe-8a4bd67cab28', 3: '476013c8-08e2-4a8a-a77a-d38c013cae34', 4: 'f3a589f1-e1f2-4a1a-8606-8fe3e89f1c53', 5: '885d405c-acd7-49f5-b05f-9831dd1da664', 6: '46639f12-9021-417c-b0b1-790fc49cb84a', 7: '527f2a2d-e5ec-4838-bd38-c5646c10d425', 8: '3a7ab908-9161-4b0b-b5e8-3da6f51ee904', 9: '1da0e5ee-20eb-417b-bf7a-b567bfa8243e', 10: 'e2700d0d-ba5d-4efe-9acb-9693355c5ee9', 11: '45b45758-dd39-4640-bc3f-e75b327808c8', 12: 'a45776e0-59dc-4f02-a8f2-52b69d32b542', 13: '11055ea8-a361-424d-b352-a37b911b9572', 14: '156125c5-0d62-492a-ad57-74c1ff035163', 15: 'd15c74f9-0285-476d-a070-595513005e16', 16: '2e8ac177-bd9f-4790-8b8a-dcb449dee108', 17: '16a9cf52-7a4a-454e-9251-b873eb246cf9', 18: '2c484411-701e-4b05-93e4-7eb4b26d1001', 19: 'a728e6fc-3db2-4e03-ab85-b7305babce2c', 20:

# 데이터 조회

In [57]:
from langchain_community.vectorstores import FAISS


# 파일 로드
db = FAISS.load_local(f'{ROOT_PATH}/vectordb/{INDEX_ID}', embeddings, allow_dangerous_deserialization=True)
    
retriever = db.as_retriever(search_kwargs={"k": 3})
query = "최고위원회 대표"
# doc = retriever.get_relevant_documents(query)
doc = retriever.invoke(
    input=query,
    config=None,  # 실행 구성을 위해 RunnableConfig 객체를 사용할 수 있음
    # 추가적인 매개변수를 kwargs로 전달 가능
)

for d in doc:
    print(d)



page_content='추경호 원내대표는 이날 오전 원내대책회의에서 "금투세, 상속세, 종부세에 대한 세제개편 문제는 더 이상 논의를 미룰 수 있는 상황이 아니다. 특히 금투세는 그냥 두면 5개월 뒤부터 시행된다. 금투세 폐지를 요구하는 국민 청원이 5만명 넘는 동의를 얻었다. 최근 주식시장의 변동성이 커지고 있는 상황을 감안하면 여야가 지금 당장 협상에 착수해야 한다"고 촉구했다.'
page_content='윤 원내대변인은 진 정책위의장이 제안한 '민생입법을 위한 양당 정책위의장 협상 테이블'에 금투세가 의제로 오를 수 있느냐는 질문에도 "그건 더 큰 문제가 아니겠나. 양당 지도부 차원에서 논의돼야 하지 않겠나"라고 답했다.'
page_content='윤종군 원내대변인은 기자들을 만나 '추경호 원내대표가 연금개혁 특위 가동 및 금투세 폐지 협상을 제안했는데 논의할 수 있나'라는 질문에 "연금개혁은 이재명 전 대표가 먼저 제안하지 않았나"라면서도 금투세 관련 질문에는 답하지 않았다.'


# code assist 기능 구현을 위한 메모

## I. vector database에 파일 내용 저장하기

1. 소스 폴더의 ROOT로 부터 모든 파일들을 읽어서 메타 정보를 수집한다.
    
    메타정보는 [경로, 파일명, 확장자, 유형등]이 있다

2. 파일을 확장자/유형 에 따라서 Split 한다.

    ex) 자바는 함수 단위, xml은 쿼리 단위

3. FAISSS등 vector database에 저장한다.

    저장된 이후 uuid등을 조회하여 메타정보와 chunk 내용을 변수에 셋팅한다. 

3. RDB의 VECTOR_DATA 테이블에 저장한다.

    위 단계에서 나온 결과를 테이블에 저장한다.



# 테이블 정보



